# RunnablePassthrough

- RunnablePassthrough를 사용하면 입력을 변경하지 않거나 추가 키를 추가하여 전달할 수 있습니다. 이는 일반적으로 맵의 새 키에 데이터를 할당하기 위해 RunnableParallel과 함께 사용됩니다.

- 단독으로 호출되는 RunnablePassthrough()는 단순히 입력을 받아 통과시킵니다.

- 할당과 함께 호출된 RunnablePassthrough(.assign(...))는 입력을 받은 후 할당 함수에 전달된 추가 인수를 추가합니다.

In [2]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{name} 이 사람은 누구야?")

prompt

PromptTemplate(input_variables=['name'], template='{name} 이 사람은 누구야?')

In [40]:
from langchain_openai import ChatOpenAI

openai_api_key = "youar_api_key_here"

model = ChatOpenAI(model = 'gpt-3.5-turbo-0125',api_key=openai_api_key)

In [4]:
chain = prompt | model

In [28]:
chain.invoke({"name":"tim cook"})

AIMessage(content='팀 쿡(Tim Cook)은 애플(Apple)의 CEO이며, 스티브 잡스(Steve Jobs)의 후계자로서 애플을 이끌고 있는 중요한 인물입니다. 그는 애플에서 1998년부터 근무하며 주요한 역할을 맡아 왔고, 2011년부터 CEO로 임명되어 애플의 성공과 혁신을 이끌어 왔습니다. 현재 애플은 세계적으로 가장 큰 기술 기업 중 하나로 자리매김하고 있습니다.', response_metadata={'finish_reason': 'stop', 'logprobs': None})

In [29]:
chain.invoke("오바마") # 에러가나는데, dict 형태로 key 값인 name : "오바마" 의 형태로 입력을 하지 않아서다.

TypeError: Expected mapping type as input to PromptTemplate. Received <class 'str'>.

In [31]:
chain_2 = {"name": RunnablePassthrough()} | prompt | model #name 이라는 key 값으로 RunnablePassthrough를 넣어준다. 
chain_2.invoke("kevin de bruyne") #invoke 에서 따로 설정없이 스트링만 입력하면 결과를 출력시킬 수 있다.

AIMessage(content='Kevin De Bruyne는 벨기에 출신의 축구 선수로 맨체스터 시티에서 뛰고 있는 중필더이다. 그는 뛰어난 패스와 슈팅 능력을 가지고 있으며, 맨체스터 시티와 벨기에 대표팀에서 중요한 역할을 하고 있는 선수이다. 현재 세계적으로도 최고의 중필더 중 한 명으로 꼽히고 있다.', response_metadata={'finish_reason': 'stop', 'logprobs': None})

In [ ]:
#RAG에서의 예제
Chain = {"document":retriver, "question": RunnablePassthrough()} | prompt | model
prompt = "{retriever}에서 {question}에 대한 답변을 찾아주세요."
#이러한 형태로 RAG 파이프라인을 구축해서 리트리버와 모델을 연결해줄 수 있다.

## 🦜examples 에서의 예제를 봐보자.

In [32]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel( #코드를 병렬로 처리할때 쓰는 런어블 패럴
    passed=RunnablePassthrough(), # 1
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3), # 1, mult =3 #assign 을 통해 값을 넣어줄 수 있다.
    modified=lambda x: x["num"] + 1,# 2
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

# RunnableParallel
RunnableParallel은 시퀀스에서 다음 런처블의 입력 형식과 일치하도록 한 런처블의 출력을 조작하는 데 유용할 수 있습니다.

여기서 프롬프트에 대한 입력은 "context" 및 "question" 키가 있는 맵이 될 것으로 예상됩니다. 사용자 입력은 질문일 뿐입니다. 따라서 리트리버를 사용하여 컨텍스트를 가져오고 "question" 키 아래의 사용자 입력을 통과시켜야 합니다.

- 일반적으로 두개의 Chain 을 묶어서 map_Chain 의 형태로 많이들 쓴다.


In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

model = ChatOpenAI(api_key=openai_api_key)
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "bear"}) #2개의 결과를 병렬로 처리할 수 있다.

{'joke': AIMessage(content="Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the distance between them!", response_metadata={'finish_reason': 'stop', 'logprobs': None}),
 'poem': AIMessage(content='In the forest deep, the bear roams free,\nA majestic creature, wild and full of mystery.', response_metadata={'finish_reason': 'stop', 'logprobs': None})}

In [14]:
prompt1 = PromptTemplate.from_template("{name}은 어디의 CEO 인가요")
prompt2 = PromptTemplate.from_template("{name}은 어떤 업적이 있나요?")

In [15]:

chain_1 = {"name": RunnablePassthrough()} | prompt1 | model
chain_2 = {"name": RunnablePassthrough()} | prompt2 | model

In [16]:
map_chain = RunnableParallel(a=chain_1, b=chain_2) #Chain 을 여러개 만들어서 묶어준다.

In [17]:
map_chain.invoke({"country":"tim cook"})

{'a': AIMessage(content="'tim cook'은 미국의 CEO이며, 미국의 기술기업인 애플(Apple)의 CEO입니다.", response_metadata={'finish_reason': 'stop', 'logprobs': None}),
 'b': AIMessage(content='"country": "tim cook"은 잘못된 입력입니다. Tim Cook은 애플(Apple)의 CEO로 유명한 인물이며, 테크놀로지 산업에서의 업적으로는 아이폰, 아이패드, 맥북 등의 제품 개발과 혁신적인 기술 발전에 기여한 것으로 알려져 있습니다.', response_metadata={'finish_reason': 'stop', 'logprobs': None})}

# RunnableLambda

파이프라인에서 임의의 함수를 사용할 수 있습니다.  
- 이러한 함수에 대한 모든 입력은 단일 인수가 되어야 한다는 점에 유의하세요. 여러 인수를 받는 함수가 있는 경우 단일 입력을 받아 여러 인수로 압축을 푸는 래퍼를 작성해야 합니다.

In [31]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


def length_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("what is {a} + {b}")
model = ChatOpenAI(api_key=openai_api_key)

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("sonny") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("sonny"), "text2": itemgetter("kangin")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

In [33]:
chain.invoke({"sonny": "kangin", "kangin": "lee-kang-in"})

AIMessage(content='6 + 66 = 72', response_metadata={'finish_reason': 'stop', 'logprobs': None})

###  Accepting a Runnable Config

실행 가능한 람다는 선택적으로 콜백, 태그 및 기타 구성 정보를 중첩된 실행에 전달하는 데 사용할 수 있는 RunnableConfig를 받을 수 있습니다.

In [37]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableConfig
import json


def parse_or_fix(text: str, config: RunnableConfig):
    fixing_chain = (
        ChatPromptTemplate.from_template(
            "Fix the following text:\n\n```text\n{input}\n```\nError: {error}"
            " Don't narrate, just respond with the fixed data."
        )
        | ChatOpenAI(api_key=openai_api_key)
        | StrOutputParser()
    )
    for _ in range(3):
        try:
            return json.loads(text)
        except Exception as e:
            text = fixing_chain.invoke({"input": text, "error": e}, config)
    return "Failed to parse"

In [39]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = RunnableLambda(parse_or_fix).invoke(
        "{soony: kangin}", {"tags": ["my-tag"], "callbacks": [cb]}
    )
    print(output)
    print(cb)

{'soony': 'kangin'}
Tokens Used: 67
	Prompt Tokens: 58
	Completion Tokens: 9
Successful Requests: 1
Total Cost (USD): $0.000105


# @Chain Decorator 

데코레이터는 기존의 Class에 기능을 추가 시킬 수 있다.

In [48]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_openai import ChatOpenAI

prompt1 = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
prompt2 = ChatPromptTemplate.from_template("What is the subject of this joke: {joke}")

@chain
def custom_chain(text):
    prompt_val1 = prompt1.invoke({"topic": text})
    output1 = ChatOpenAI().invoke(prompt_val1)
    parsed_output1 = StrOutputParser().invoke(output1)
    chain2 = prompt2 | ChatOpenAI() | StrOutputParser()
    return chain2.invoke({"joke": parsed_output1})

custom_chain.invoke("bears")

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)